<h1> Monte Carlo simulation - limb darkening in plane-parallel, grey atmospheres </h1>


Limb darkening describes the fact that a (normal) star seems to be darker at the edge than at its center. This can be clearly observed for the sun. If, as in Example 1.13, the direction cosine (i.e., the cosine of angle between radius vector and photon direction) is denoted by $\mu$, one can approximiate the angular dependency of the specific intensity (see below) by the expression

$$I(\mu) ≈ I_1(0.4 + 0.6 \mu) \quad\quad\quad\quad(1)$$

where $I_1$ is the specific intensity for the radial direction, $\mu = 1$, i.e., the intensity observed at the center of a star. For $\mu = 0$, on the other hand, the angle between radial and photon’s direction is 90$^{\circ}$  (this condition is met at the limb of the star), and the corresponding region appears to be fainter, by a factor of roughly 0.4.

Eq.(1) is the consequence of an approximate solution of the equation of radiative transfer in plane-parallel symmetry (i.e., under the condition that the stellar photosphere is very thin compared to the stellar radius: the solar photosphere, e.g., is only a few hundred kilometers thick, contrasted to the sun’s radius of 700 000 km), with absorption and emission processes assumed to be grey, i.e., frequency independent, which has been developed  by Eddingtion and Barbier. 
Note that the approximate solution of Milne’s integral equation (Example 1.13) is based on the same approach. Just one additional comment: The above relation (1) has nothing to do with the presence of any temperature stratification (though it can be influenced by it), but is the consequence of a large number of absorption and emission processes in an atmosphere of finite (optical) depth, as we will see below.

In order to avoid almost all subtleties of radiative transfer and corresponding approximate solutions, in this exercise we will perform a Monte Carlo simulation to confirm the above result. The principle strategy is very close to the simulation as described in Example 1.13, but we will sort the photons according to the direction they leave the atmosphere, and not according to their energy.



In [3]:
import math
import random
import numpy as np
import time
%matplotlib qt
import matplotlib.pyplot as plt
import scipy.optimize
from styles.matplotlib_style import *

<h3>a) Develop the Algorithm to Simulate Photons in an Atmosphere </h3>

Develop a sketch of the program flow including all possible branches (a so-called flow-chart), accounting for appropriate distributions of emission angle (Example 1.18), optical path length and scattering angle. Update always the radial optical depth of the photon, according to the comments in the program, and follow the photons until they have left the atmosphere. In the latter case then, update the corresponding array counting the number of photons which have escaped under a certain range of angles. Before implementing this program, discuss your flow-chart with your supervisor.

Develop an algorithm to simulate the propagation of photons in an atmosphere and write it down in the form of a flow-chart. Your algorithm should include all possible branches, accounting for appropriate distributions of emission angle (Example 1.18), optical path length and scattering angle. Update always the radial optical depth of the photon and propagate the photons until they have left the atmosphere. Once a photon as left the atmosphere, return the corresponding directional cosine $\mu$ under which the photon was emitted.
<b>Before implementing this program, discuss your flow-chart with your supervisor.</b>

Note that tau and tau_max are *radial* optical depths.
Since photons have a direction of mu with respect to the radial
direction, we have to account for projection effects.
In order to simulate the photons, along their way through the
atmosphere you have to decide whether a photon is
- inside the atmoshere (0 < tau < tau_max)
- has left the atmosphere (tau < 0)
- or has been scattered back into the stellar core (tau > taumax)
and choose the appropriate next step for the photon.

If a photon was found to have been scattered back into the core,
just release a new photon at the inner boundary.


<h3>b) Implement the Algorithm to Simulate Photons in an Atmosphere</h3>

In this step you should implement the algorithm you developed in a)
and test it for $10^4\dots10^5$ photons and an optical depth of the atmosphere of tau_max=10.
From the $\mu$ values returned from your code, build a histogram.
<b> Implement the histogram function yourself. For this exercise, do not use a histogram function from a library.</b>
But it's okay to use the numpy histogram function for testing. Just make sure you use your own histogram function in the end.

Using 20 channels and make a plot like this one:

<img src="https://dmz-sv-owncloud.physik.uni-muenchen.de/index.php/apps/files_sharing/ajax/publicpreview.php?x=2864&y=1580&a=true&file=Limb_N.png&t=d3c6c60bc6af21213afcc775a2622a2c&scalingup=0" width="600">



In [39]:
from simulate_rte import simulate
import time

start = time.time()
mu_store = simulate(num_of_photons=int(1e5), tau_max=0.001)
end = time.time()

print(f"Simulation took {start-end} seconds.")

Simulation took -0.32138991355895996 seconds.


In [40]:
mu_store = np.float64(mu_store)

theta_in_deg = np.arccos(mu_store)*360/(2*np.pi)
bin_width_mu = 0.05
bin_width_theta = 4
bin_spacing_mu = np.arange(0, max(mu_store) + bin_width_mu, bin_width_mu)
bin_spacing_theta = np.arange(0, max(theta_in_deg) + bin_width_theta, bin_width_theta)

plt.subplot(2, 1, 1)
frequency_and_edges = plt.hist(mu_store, histtype="step", color=blue, lw=3, bins=bin_spacing_mu)
frequency_mu = frequency_and_edges[0]

plt.xlabel(r"$\mu$ value ")
plt.ylabel("$N(\mu)$")
plt.xlim(-0.05,1.05)
plt.ylim(-100,130000)

# For Plotting the mid-point-intervals of N(μ), remove the last element of the bin spacing of the mu histogram
bin_spacing_mu = bin_spacing_mu[:-1]
mu_midpoints = bin_spacing_mu+bin_width_mu/2
plt.plot(mu_midpoints, frequency_mu, ".", color="DodgerBlue", markersize=15, )


plt.subplot(2, 1, 2)
plt.hist(theta_in_deg, histtype="step", color=red, lw=3, bins=bin_spacing_theta)
plt.xlabel(r"$\vartheta$ value $[^{\circ}]$")
plt.ylabel(r"$N(\vartheta)$")
plt.xlim(0,100)


#theta = np.arccos(mu_store_test)/2  # Frage ali warum
# plt.hist(np.arccos(mu_store_test), bins="auto")
# Passt, Ali fragen, irgebdwas mit pi/2 soll null
# plt.vlines(np.pi/2, 0, 2600, label="pi/2 lineo ")
plt.tight_layout()
# plt.savefig("figures/4_distribution_of_final_mu_after_simulation_10_tpw_5_photons_tau_max_10.png")
plt.show()



In [18]:

plot_normalized_mu_distribution = False
if plot_normalized_mu_distribution:
    plt.plot(mu_midpoints, frequency_mu/frequency_mu[-1]/mu_midpoints, ".", color=blue)
    plt.plot(mu_midpoints, mu_midpoints, "--", color="black")
    plt.ylabel("")
    plt.xlabel("$\mu$")


In [23]:
# use the following code to visualize the data, adjust the number of bins and photons
n_photons = 100
n_bins = 5
mu_values = mu_store
bins = np.linspace(0,1,n_bins) # those are the bin edges
n_mu, _ = np.histogram(mu_values, bins=bins)
n_mu_plot = np.concatenate(([0], n_mu))
plt.step(bins, n_mu_plot, color='DodgerBlue')
plt.xlabel('$\mu$')
plt.ylabel('$N(\mu)/N(1)$');

<h3>c) Comparison to Equation (1)</h3>



In our simulation, we have calculated the *number* of photons leaving the atmosphere with respect to a surface perpendicular to the radial direction. Without going into details, this number is proportional to the specific intensity weighted with the projection angle μ, since the specific intensity, I(μ), is defined with respect to unit projected area. 

$$ N(\mu){\rm d}\mu \propto I(\mu)\mu{\rm d}\mu $$

and the intensity within dμ is obtained from the number of photons, divided by an appropriate average of μ, i.e., centered at the mid of the corresponding channel.

This relation is also the reason why the distribution of the photons with respect to μ at the lower boundary follows the pdf μdμ: for an isotropic radiation field, which is assumed to be present at the lowermost boundary, it is the specific intensity and not the photon number which is uniformly distributed with respect to μ! 

Thus, in order to convert to photon numbers, we have to draw the emission angles at the lower boundary from the pdf μdμ instead of dμ! Inside the atmosphere, on the other hand, the emission angle refers to the photons themselves and thus is (almost) isotropic, so that we have to draw from the pdf dμ.

Write a function/cell to display the specific intensity and compare with the prediction Eq. (1). Use $N = 10^6$ photons for tau_max = 10 now, and derive the limb-darkening coefficients (in analogy to Eq. (1)) from a linear regression to your results. 

Use scipy.optimize.curve_fit to determine the coefficients - see the scipy documentation how to use it http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.optimize.curve_fit.html



In [41]:
def linear(x, a, b):
    return x*a+b

mu_theoretical = np.linspace(1e-5, 1, 1000)
I_theoretical = linear(mu_theoretical, 0.6, 0.4)

frequency_mu_normalized = frequency_mu/frequency_mu[-1]

from scipy.optimize import curve_fit
popt = curve_fit(linear, mu_midpoints, frequency_mu_normalized/mu_midpoints)
opt_a = popt[0][0]
opt_b = popt[0][1]
sigma_a = np.sqrt(popt[1][0][0])
sigma_b = np.sqrt(popt[1][1][1])

print("Best fitted values: ", opt_a, opt_b, "Relative errors: ", sigma_a/opt_a, sigma_b/opt_b)

gaussian_error_on_line = np.sqrt((sigma_a*mu_theoretical) ** 2 + sigma_b ** 2)

plt.title("Statistical M.C. vs. theoretical approximation")
plt.plot(mu_midpoints, frequency_mu_normalized/mu_midpoints, ".", markersize=15, color="DodgerBlue", label=r"$N(\mu)/(\mu\cdot N(1))$")
plt.plot(mu_theoretical, I_theoretical, "--", color="black", label="Theoretical approximation", lw=3)

# Error lines
lightest_green = (0.23, 0.85, 0.25, 0.2)
plt.errorbar(x=mu_theoretical, y=linear(mu_theoretical, opt_a, opt_b), yerr=gaussian_error_on_line, ecolor=lightest_green, color=green, lw=1,  label=rf"Fit on $N(\mu)/(\mu \cdot N(1))$: ")

plt.legend()
plt.ylabel(r"Intensity $I$")
plt.xlabel(r"$\mu$")
plt.show()

plt.savefig("figures/6_final_intensity_fit_tau_0_001_N_photons_10^5")



Best fitted values:  0.014078303858682473 1.0266750306502763 Relative errors:  0.8657006992280968 0.00685154769569403


<h3>d) Limiting case tau_max << 1 </h3>

To convince yourself that limb-darkening is the consequence of a multitude of scattering effects, reduce tau_max to a very small value. Which angular distribution do you expect now for the specific intensity? Does your simulation verify your expectation?

In [ ]:
# put your code here

plt.plot(mu_plot, 0.4+0.6*mu_plot, 'k-', label='Analytic approx.', alpha=0.5);
# plt.plot(mu_bin_centers,I_mu_thin, 'o', color='DodgerBlue', label='Monte Carlo')
plt.xlabel('$\mu$')
plt.ylabel('$I(\mu)/I(1)$')
plt.legend(loc='lower right')
